# 해보자일단

In [138]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd

In [159]:
feature_name_df = pd.read_csv('../00.data/UCI HAR Dataset/UCI HAR Dataset/features.txt', sep='\s+', header=None, 
                              names=['column_index','column_name'])

In [140]:
feature_name_df.head()

,column_index,column_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y


In [141]:
feature_name_df.column_name

0                         tBodyAcc-mean()-X
1                         tBodyAcc-mean()-Y
2                         tBodyAcc-mean()-Z
3                          tBodyAcc-std()-X
4                          tBodyAcc-std()-Y
                       ...                 
556        angle(tBodyGyroMean,gravityMean)
557    angle(tBodyGyroJerkMean,gravityMean)
558                    angle(X,gravityMean)
559                    angle(Y,gravityMean)
560                    angle(Z,gravityMean)
Name: column_name, Length: 561, dtype: object

In [142]:
len(feature_name_df.column_name.unique())

477

### 모두 다른 이름을 갖고 있는게 아니다

In [143]:
unique_name = feature_name_df.column_name.unique()
for i in unique_name:
    if len(feature_name_df.column_name[feature_name_df.column_name == i]) > 1 :
        count = len(feature_name_df.column_name[feature_name_df.column_name == i])
        print (f' 갯수 :{count}')
        print (f' 칼럼이름 : {i}')

 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-1,8
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-9,16
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-17,24
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-25,32
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-33,40
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-41,48
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-49,56
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-57,64
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-1,16
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-17,32
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-33,48
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-49,64
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-1,24
 갯수 :3
 칼럼이름 : fBodyAcc-bandsEnergy()-25,48
 갯수 :3
 칼럼이름 : fBodyAccJerk-bandsEnergy()-1,8
 갯수 :3
 칼럼이름 : fBodyAccJerk-bandsEnergy()-9,16
 갯수 :3
 칼럼이름 : fBodyAccJerk-bandsEnergy()-17,24
 갯수 :3
 칼럼이름 : fBodyAccJerk-bandsEnergy()-25,32
 갯수 :3
 칼럼이름 : fBodyAccJerk-bandsEnergy()-33,40
 갯수 :3
 칼럼이름 : fBodyAccJerk-bandsEnergy()-41,48
 갯수 :3
 칼럼이름 : fBodyAccJerk-bandsEnergy()-49,56
 갯수 :3
 칼럼이름 : fBodyAccJerk-bandsEnergy()-57,64
 갯수 :3
 

### 같은 이름을 갖고 있는 피쳐

In [169]:
for i in unique_name:
    if len(feature_name_df.column_name[feature_name_df.column_name == i]) > 1 :
        t_l = feature_name_df.column_name[feature_name_df.column_name == i].index
        feature_name_df.column_name[feature_name_df.column_name == i] += ['_1','_2','_3']

In [170]:
len(feature_name_df.column_name.unique())

561

#### 각각 다른이름 붙인 것을 확인

In [171]:
feature_name = feature_name_df.iloc[:, 1].values.tolist()

### 데이터 불러오기

In [173]:
X_train = pd.read_csv('../00.data/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt', sep='\s+', names=feature_name)
X_test = pd.read_csv('../00.data/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt', sep='\s+', names=feature_name)

In [177]:
y_train = pd.read_csv('../00.data/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt', sep='\s+', names=['action'])
y_test = pd.read_csv('../00.data/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt', sep='\s+', names=['action'])

In [178]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7352, 561), (7352, 1), (2947, 561), (2947, 1))

### 제약 x

In [183]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)

In [184]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.8547675602307431

### 최적의 하이퍼 파라미터

In [187]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth':[ 6, 8 ,10, 12],
    'min_samples_leaf':[5,6,7,8],
    'min_samples_split':[2,3,4] 
}
grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)

print('GridSearchCV 최적 하이퍼 파라미터:', grid_dclf.best_params_)
print(f'GridSearchCV 최고 정확도: {grid_dclf.best_score_:.4f}')
best_dclf = grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
pred = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print(f'테스트 세트에서의 DecisionTreeClassifier 정확도: {accuracy:.4f}')

GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 10, 'min_samples_leaf': 6, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.8556
테스트 세트에서의 DecisionTreeClassifier 정확도: 0.8748


### 중요도

In [188]:
import seaborn as sns
%matplotlib inline

In [214]:
imp = pd.DataFrame(best_dclf.feature_importances_, index = feature_name)

In [215]:
imp.head()

,0
tBodyAcc-mean()-X,0.000000
tBodyAcc-mean()-Y,0.000000
tBodyAcc-mean()-Z,0.000000
tBodyAcc-std()-X,0.000000
tBodyAcc-std()-Y,0.001014


In [218]:
chart = imp.sort_values(by=0,ascending=False)

array([[0.24767659],
       [0.2078194 ],
       [0.13723939],
       [0.11289052],
       [0.09927213],
       [0.02343368],
       [0.01683346],
       [0.01606515],
       [0.0123295 ],
       [0.00896362],
       [0.00881301],
       [0.0086268 ],
       [0.00614936],
       [0.00524993],
       [0.00512918],
       [0.00427487],
       [0.00387784],
       [0.00339332],
       [0.00305895],
       [0.00295676]])

In [223]:
sns.barplot(x = chart[:20].values, y = chart[:20].index)

Exception: Data must be 1-dimensional

In [224]:
chart[:20].index

Index(['tGravityAcc-min()-X', 'fBodyAccJerk-bandsEnergy()-1,16_1',
       'angle(Y,gravityMean)', 'fBodyAccMag-energy()',
       'tGravityAcc-arCoeff()-Z,2', 'fBodyGyro-maxInds-X',
       'tBodyGyro-correlation()-Y,Z', 'tGravityAcc-energy()-Y',
       'tBodyAccMag-arCoeff()1', 'tBodyGyro-max()-X',
       'tBodyAcc-correlation()-X,Y', 'fBodyAcc-max()-Y', 'tGravityAcc-max()-Y',
       'tGravityAcc-mean()-Y', 'fBodyGyro-max()-X', 'tBodyGyro-min()-X',
       'tBodyAccJerk-correlation()-X,Y', 'tBodyGyroJerk-max()-X',
       'tGravityAccMag-arCoeff()1', 'tGravityAcc-arCoeff()-Y,3'],
      dtype='object')

In [226]:
chart[:20].values

array([[0.24767659],
       [0.2078194 ],
       [0.13723939],
       [0.11289052],
       [0.09927213],
       [0.02343368],
       [0.01683346],
       [0.01606515],
       [0.0123295 ],
       [0.00896362],
       [0.00881301],
       [0.0086268 ],
       [0.00614936],
       [0.00524993],
       [0.00512918],
       [0.00427487],
       [0.00387784],
       [0.00339332],
       [0.00305895],
       [0.00295676]])